#Covid detection using cnn

In [1]:
#Allow google drive access

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#set path

TRAIN_PATH="/content/drive/My Drive/Colab Notebooks/covid_19/CovidDataset/Train"
VAL_PATH="/content/drive/My Drive/Colab Notebooks/covid_19/CovidDataset/Val"

In [28]:
#importing libraries we need

import os
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.models import Sequential

In [4]:
##BUILDING  CNN based model in KERAS ( we will create 4 cnn layers and then multiple classification layer)

model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))         #ADD, 32 feature extractor in 1st layer , default kernel size,size ou is 224*224 and 3 channel(colour=RGB)
model.add(Conv2D(64,(3,3),activation='relu'))                                             #ADD another layer of filters
model.add(MaxPooling2D(pool_size=(2,2)))                                                  #add max pooling layer default size is 2,2
model.add(Dropout(0.25))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

#ADD Flatten and dense layer
model.add(Flatten())                                                                       #means we are making an array of pixels that represent an image
model.add(Dense(64,activation='relu'))                                                      #
model.add(Dropout(0.5))                                                                    #we are dropping 50% neuron so to decrease complexity & prevent overfitting
model.add(Dense(1,activation="sigmoid"))

#using 1 neuron cause it is a binary classification
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])

In [5]:
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [6]:
#Train from scratch
train_datagen=image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)
test_dataset = image.ImageDataGenerator(rescale=1./255)


In [10]:
#for train 
train_generator=train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/CovidDataset/Train',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 224 images belonging to 2 classes.


In [11]:
#for validation 
validation_generator=train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/CovidDataset/Val',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 60 images belonging to 2 classes.


In [17]:

history=model.fit_generator(
    train_generator,
    steps_per_epoch=5,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=2
)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
5/5 [==============================] - 9s 2s/step - loss: 0.2361 - accuracy: 0.9062 - val_loss: 0.2249 - val_accuracy: 0.9500
Epoch 2/10
5/5 [==============================] - 8s 2s/step - loss: 0.2657 - accuracy: 0.8687 - val_loss: 0.2167 - val_accuracy: 0.9167
Epoch 3/10
5/5 [==============================] - 8s 2s/step - loss: 0.3363 - accuracy: 0.8875 - val_loss: 0.3832 - val_accuracy: 0.9667
Epoch 4/10
5/5 [==============================] - 8s 2s/step - loss: 0.2682 - accuracy: 0.9125 - val_loss: 0.1553 - val_accuracy: 0.9667
Epoch 5/10
5/5 [==============================] - 8s 2s/step - loss: 0.1707 - accuracy: 0.9500 - val_loss: 0.1290 - val_accuracy: 0.9833
Epoch 6/10
5/5 [==============================] - 8s 2s/step - loss: 0.1377 - accuracy: 0.9563 - val_loss: 0.0984 - val_accuracy: 0.9500
Epoch 7/10
5/5 [==============================] - 9s 2s/step - loss: 0.1336 - accuracy: 0.9438 - val_loss: 0.1337 - val_accuracy: 0.9500
Epoch 8/10
5/5 [=========================

In [ ]:
#Showing accuracy of train and validation

In [18]:
model.evaluate_generator(train_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.09757165610790253, 0.9732142686843872]

In [19]:
model.evaluate_generator(validation_generator)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1877: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  warnings.warn('`Model.evaluate_generator` is deprecated and '


[0.08145670592784882, 0.9833333492279053]